In [4]:
import requests
import pandas as pd
import datetime
import pytz
import json

from agol_restapi_tools import create_log_ids
from agol_restapi_tools import agol_date_convert_akt
from agol_restapi_tools import columns_compare
from agol_restapi_tools import pd_to_attributes_list
from agol_restapi_tools import token_generation
from agol_restapi_tools import agol_table_to_pd
from agol_restapi_tools import add_new_logs
from agol_restapi_tools import oid_field
from agol_restapi_tools import locate_objectid

token = token_generation("AKDOT_COMMUNICATIONS", "Skisnowbird1")

### Pull Master Project Data

In [39]:
projects_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/CY2023_Project_Polygons_notcleaned/FeatureServer"
projects = agol_table_to_pd(projects_url, 0, token, drop_objectids="Y")

### Re-Load Start and End Points

In [ ]:
starts_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/Start_Points/FeatureServer"
ends_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/End_Points/FeatureServer"

In [ ]:
# Clear Starts Points from Layer
delete_url = f"{starts_url}/0/deleteFeatures"

params = {
    'f':'json',
    'where':'1=1',
    'token':token
}

response = requests.post(delete_url, params)

In [ ]:
# Clear Ends Points from Layer
delete_url = f"{ends_url}/0/deleteFeatures"

params = {
    'f':'json',
    'where':'1=1',
    'token':token
}

response = requests.post(delete_url, params)

In [ ]:
counter = 0

# Iterate Through Projects UIDS, Pull Geometry, Add Entry to Start and End with Lat, Long and Project Data
for index, row in projects.iterrows():

    uid = row["Unique_ID"]
    name = row['Project_Name']
    iris = row['IRIS_Number']

    if name == None:
        name = ""
        
    if iris == None:
        iris = ""

    query = f"{projects_url}/0/query"

    params = {
        'where': f'Unique_ID={uid}',
        'outFields': '*',
        'returnGeometry': 'true',
        'f': 'geojson',
        'token':token
    }

    response = requests.get(query, params)

    # Extract the geometry from the response
    data = response.json()
    geometry = [feature['geometry'] for feature in data['features']]

    if geometry[0]['type'] == "MultiPolygon":
        geometry[0]['coordinates'] = geometry[0]['coordinates'][0]

    #Extract Min and Max Lat,Lng
    min = geometry[0]['coordinates'][0][0]
    min_lat= min[1]
    min_lng = min[0]

    max = geometry[0]['coordinates'][0][int(len(geometry[0]['coordinates'][0])/2)]
    max_lat= max[1]
    max_lng = max[0]


   #Create Start and End Payloads
    start_geo_payload =  [
        {
            "geometry": {
            "x": min_lng,
            "y": min_lat,
                'spatialReference': {
                'wkid': 4326 }
            },
            "attributes": {
            "UID": uid,
            "Project_Name":name,
            "IRIS_Number":iris
            } 
        } 
        ]

    end_geo_payload =  [
        {
            "geometry": {
            "x": max_lng,
            "y": max_lat,
                'spatialReference': {
                'wkid': 4326 }
            },
            "attributes": {
            "UID": uid,
            "Project_Name":name,
            "IRIS_Number":iris
            } 
        } 
        ]
    

    #Load Start to Layer
    start_update_url = f'{starts_url}/0/applyEdits'

    start_update_params = {
        'f':'json',
        'token':token,
        'adds':f'{start_geo_payload}'
    }

    start_response = requests.patch(url = start_update_url, params = start_update_params)
    print(start_response.json())


    #Load End to Layer
    end_update_url = f'{ends_url}/0/applyEdits'

    end_update_params = {
        'f':'json',
        'token':token,
        'adds':f'{end_geo_payload}'
    }

    end_response = requests.patch(url = end_update_url, params = end_update_params)
    print(end_response.json())

    counter += 1

### Re-Load the Alaksa 511 Management Table

In [2]:
management_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/AK_511_Management/FeatureServer"

In [41]:
# Clear Starts Points from Layer
delete_url = f"{management_url}/0/deleteFeatures"

params = {
    'f':'json',
    'where':'1=1',
    'token':token
}

response = requests.post(delete_url, params)

In [44]:
#Grab Subset of Project Data Needed for Project Input
project_sel = projects[[
    "Unique_ID",
    "Project_Name",
    "IRIS_Number",
    "Route_ID",
    "Route_Name",
    "Federal_Project_Number",
    "STIP_ID",
    "DOT_PF_Region",
    "Borough_CensusArea",
    "State_House_District",
    "State_Senate_District",
    "Project_Description",
    "Project_Purpose",
    "Project_Website",
    "Project_Priority_Area",
    "Project_Engineer_Email",
    "Design_Engineer",
    "Design_Engineer_Phone",
    "Construction_Engineer",
    "Construction_Engineer_Phone",
    "Contract_Value",
    "Anticipated_Start",
    "Anticipated_End",
    "Funding_Type",
    "Project_Practice",
    "Project_Location",
    "Project_Limit",
    "Project_Impact",
    "Construction_Description",
    "Longitude",
    "Latitude"
]]


#Rename Unique ID Field
project_sel.rename(columns = {"Unique_ID":"UID"}, inplace = True)

#Projects Fill Missing Values
project_sel.fillna("", inplace = True)

#Replace Empty String Characters
project_sel.replace('\xa0', '', regex=True, inplace = True)



#Iterate Through Project Select Entries, Create Entry Row and Add to Feature Layer
for index, row in project_sel.iterrows():
    df = pd.DataFrame(row).T
    entry = pd_to_attributes_list(df)

    applyEdits = f"{management_url}/0/applyEdits"

    params = {
        "f":'json',
        "token":token,
        "adds":f'{entry}'
        }

    print(requests.post(applyEdits, params).json())


{'addResults': [{'objectId': 588, 'uniqueId': 588, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
{'addResults': [{'objectId': 589, 'uniqueId': 589, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
{'addResults': [{'objectId': 590, 'uniqueId': 590, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
{'addResults': [{'objectId': 591, 'uniqueId': 591, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
{'addResults': [{'objectId': 592, 'uniqueId': 592, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
{'addResults': [{'objectId': 593, 'uniqueId': 593, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
{'addResults': [{'objectId': 594, 'uniqueId': 594, 'globalId': None, 'success': True}], 'updateResults': [], 'deleteResults': []}
{'addResults': [{'objectId': 595, 'uniqueId': 595, 'globalId': None, 'success': True}], 'u